In [10]:
import numpy as np
from scipy import ndimage
from scipy.spatial.transform import Rotation as R

target =    [3, 1, 0] # anchor vector connecting centroid and anchor point
reference = [1, 0, 0]    # unit vector along x axis

for _ in range(1):

    rotation_matrix = R.align_vectors(np.array([reference]), np.array([target]))[0].as_matrix()
    rotation_matrix @ np.array(target)
    # R.align_vectors(np.array([reference]), np.array([target]))[0].as_rotvec() * 180 / np.pi

    print(rotation_matrix @ np.array(target))

[3.16227766e+00 1.11022302e-16 0.00000000e+00]


In [11]:
target =    np.array([[0, 3, 0],
                      [0, 1, 0],
                      [0, 0, 4]])


reference = [1, 0, 0]    # unit vector along x axis

old_mat = np.diag(np.ones(3))

print(old_mat)


rotation_matrix = R.align_vectors(np.array([reference]), np.array([target[1]]))[0].as_matrix()
mat = old_mat @ rotation_matrix
r = np.array([mat @ v for v in target])
r

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


array([[3., 0., 0.],
       [1., 0., 0.],
       [0., 0., 4.]])

In [12]:
a = np.array([0,1,0])
b = np.array([0,0,3])

print(a)
np.add(a, b, out=a, casting='unsafe')
print(a)

[0 1 0]
[0 1 3]


In [13]:
rotations = R.align_vectors(np.array([reference]), np.array([target]))[0].as_euler('xyz')*(360/(2*np.pi))
rotations
# positive rotations are always anti-clockwise

ValueError: Expected input `b` to have shape (N, 3), got (1, 3, 3).

In [ ]:
n = 3
scalar_field = np.arange(n**3).reshape((n,n,n))

for axis, angle in enumerate(rotations):
    axes = [0,1,2]
    axes.remove(axis)
    scalar_field = ndimage.rotate(scalar_field, angle, axes=tuple(axes), reshape=False)




print(scalar_field)

In [ ]:
def _rotate_scalar_field(array, rotations):
    '''
    :params array:      an input scalar field with shape (a,b,c)
    :params rotations:  tuple of shape (1,3) with rotation 
                        angles along x, y, z axes, in degrees
    :params pivot:      pivot of the rotation
    :return:            rotated array
    '''
    for axis, angle in enumerate(rotations):
        axes = [0,1,2]
        axes.remove(axis)
        array = ndimage.rotate(array, angle, axes=tuple(axes), reshape=False)
    return array


In [ ]:
def _orient_along_x(array, vector):
    '''
    :params array:    array of atomic coordinates arrays: len(array) structures with len(array[i]) atoms
    :params vector:   list of shape (1,3) with anchor vector to align to the x axis
    :return:          array, aligned so that vector is on x
    '''
    assert vector.shape[1] == 3
    if vector.shape[0] > 1:
        vector = np.mean(vector, axis=0)
    rotation_matrix = R.align_vectors(np.array([[1,0,0]]), np.array([vector]))[0].as_matrix()
    return np.array([rotation_matrix @ v for v in array])

In [ ]:
vectors = np.array([[0,1,0],
                    [0,4,0],
                    [1,0,0],
                    [0,0,9],
                    [1,1,1]])
_orient_along_x(vectors, vectors[0])

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

def _orient_along_x(array, vector):
    '''
    :params array:    array of atomic coordinates arrays: len(array) structures with len(array[i]) atoms
    :params vector:   list of shape (1,3) with anchor vector to align to the x axis
    :return:          array, aligned so that vector is on x
    '''
    rotation_matrix = R.align_vectors(np.array([[1,0,0]]), np.array([vector]))[0].as_matrix()
    return np.array([rotation_matrix @ v for v in array])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

vectors = np.array([[0,1,0],
                    [1,0,0],
                    [0,0,1]])

x = [v[0] for v in vectors]
y = [v[1] for v in vectors]
z = [v[2] for v in vectors]

for i in range(len(x)):
    ax.plot([0, x[i]], [0, y[i]], [0, z[i]], label='original', color='r')

vectors = _orient_along_x(vectors, vectors[0])

x = [v[0] for v in vectors]
y = [v[1] for v in vectors]
z = [v[2] for v in vectors]

for i in range(len(x)):
    ax.plot([0, x[i]], [0, y[i]], [0, z[i]], label='rotated', color='b')

ax.legend()
plt.show()

In [ ]:
a = np.arange(3**3).reshape((3,3,3))
_rotate_scalar_field(a, np.array([45,0,0]))

In [ ]:
R.from_rotvec(np.array([90,0,0]) / 180 * np.pi).as_matrix()

In [14]:
def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)


a = np.array([1,2])
b = np.array([3,4])

cartesian_product(a,b)

array([[1, 3],
       [1, 4],
       [2, 3],
       [2, 4]])

In [ ]:
a = np.identity(3) * 2
b = np.random.rand(9).reshape((3,3))
print(b)

print(a @ b)

print(b @ a)

In [ ]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [ ]:
from rmsd import kabsch
import numpy as np

a = np.array([[0,-1,0],
              [1,0,0],
              [2,-1,0]])

b = np.array([[2,1,0],
              [2,2,0],
              [3,2,0]])

indexes = [1,2]

aa = a - centroid(a[indexes])
bb = b - centroid(b[indexes])

r = kabsch(aa[indexes], bb[indexes])

# aa = np.array([r @ v for v in aa])
bb = np.array([r @ v for v in bb])

import matplotlib.pyplot as plt

xa = [v[0] for v in a]
ya = [v[1] for v in a]

xaa = [v[0] for v in aa]
yaa = [v[1] for v in aa]

xb = [v[0] for v in b]
yb = [v[1] for v in b]

xbb = [v[0] for v in bb]
ybb = [v[1] for v in bb]

fig = plt.figure()
plot = plt.scatter(xa, ya, color='b', marker='s', linewidths=10)
plot = plt.scatter(xb, yb, color='b', marker='s', linewidths=10)
plot = plt.plot(xa, ya, color='b')
plot = plt.plot(xb, yb, color='b')

plot = plt.scatter(xaa, yaa, color='r', marker='s', linewidths=10)
plot = plt.scatter(xbb, ybb, color='r', marker='s', linewidths=10)
plot = plt.plot(xaa, yaa, color='r')
plot = plt.plot(xbb, ybb, color='r')


plt.show()

In [25]:
def ciao(*args):
    print(args)
    return np.prod([args])

ciao(*[i for i in range(1,8)])

(1, 2, 3, 4, 5, 6, 7)


5040

In [ ]:
def rot_mat_from_pointer(pointer, angle):
    '''
    Returns the rotation matrix that rotates a system around the given pointer
    of angle degrees. The algorithm is based on scipy quaternions.
    :params pointer: a 3D vector
    :params angle: a int/float, in degrees
    :return rotation_matrix: matrix that applied to a point, rotates it along the pointer
    '''
    assert pointer.shape[0] == 3

    pointer = norm(pointer)

    quat = np.array([np.sin(angle/2)*pointer[0],
                    np.sin(angle/2)*pointer[1],
                    np.sin(angle/2)*pointer[2],
                    np.cos(angle/2)])            # normalized quaternion, scalar last (i j k w)

    return R.from_quat(quat).as_matrix()